## 📝 **Manual: Running the Emotional AI Flask App**

This guide will walk you through the steps to set up and run a Colab notebook that uses a Flask API, Pyngrok, and the Gemini API to create a conversational AI that responds to user emotions.

### ⚙️ **Prerequisites and Setup**

Before you can run the code, you'll need to set up two crucial API keys.

1.  **Get a Gemini API Key:**
    * Go to the **Google AI for Developers** website.
    * Sign in and navigate to the **Get API Key** section.
    * Create a new API key and copy it. This key allows your code to access the Gemini large language model.

2.  **Get an Ngrok Authtoken:**
    * Go to the **Ngrok website**.
    * Sign up for a free account.
    * After signing in, go to the **"Your Authtoken"** section on the dashboard.
    * Copy the authtoken. This token authenticates your account, allowing you to create a secure tunnel from your Colab notebook to the internet.

---

### 💻 **Step-by-Step Instructions**

1.  **Open a New Google Colab Notebook:**
    * Go to **Google Colab**.
    * Click on **"File"** > **"New notebook"**.

2.  **Add Your API Keys to Colab Secrets:**
    * On the left-hand side of your Colab notebook, find the **key icon** (🔒) that says **"Secrets"**.
    * Click on it.
    * Click **"+ New secret"**.
    * For the first secret, set the **Name** as `ngrok_key` and paste your Ngrok authtoken in the **Value** field.
    * For the second secret, set the **Name** as `gemini_api_key` and paste your Gemini API key in the **Value** field.
    * Make sure to **enable** the "Notebook access" toggle for both secrets. 

3.  **Paste and Run the Code:**
    * Copy the entire code snippet you provided.
    * Paste it into a single code cell in your Colab notebook.
    * Click the **play button** (▶️) or press `Shift + Enter` to run the cell.

4.  **Monitor the Output:**
    * The first time you run it, the notebook will install the required libraries (`flask`, `pyngrok`, `transformers`, `torch`, and `google-generativeai`).
    * It will then connect to Ngrok, which will generate a public URL. You'll see an output that looks like: `Public URL: https://[random-string].ngrok-free.app`.
    * This URL is your live API endpoint. It's temporary and will change each time you run the notebook.

---

### 🧪 **How to Test the API**

Once the notebook is running and you have the public URL, you can test your API endpoint using various methods.

#### **Method 1: Using `curl` (Command Line)**

Open a terminal or command prompt and replace the URL with your own public URL.

```bash
curl -X POST \
  -H "Content-Type: application/json" \
  -d '{"text": "I feel so happy today, I got a new job!"}' \
  https://[your-ngrok-url].ngrok-free.app/predict

In [ ]:
!pip install flask pyngrok transformers torch

import os
import torch
import torch.nn.functional as F
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import google.generativeai as genai
from google.colab import userdata

try:
    __import__('google.generativeai')
except ImportError:
    !pip install -q google-generativeai

emo_tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
emo_model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")

In [ ]:
try:
    NGROK_AUTHTOKEN = userdata.get('ngrok_key')
    GEMINI_API_KEY = userdata.get('gemini_api_key')
    if not NGROK_AUTHTOKEN:
        raise ValueError("ngrok_key not found in userdata. Please add it.")
    if not GEMINI_API_KEY:
        raise ValueError("gemini_api_key not found in userdata. Please add it.")

    !ngrok config add-authtoken $NGROK_AUTHTOKEN

    genai.configure(api_key=GEMINI_API_KEY)
except ValueError as e:
    print(f"Error: {e}")
    exit()

app = Flask(__name__)

In [3]:
@app.route("/predict", methods=["POST"])
def predict():
    text = request.json.get("text", "")

    if not text:
        return jsonify({"error": "No text provided"}), 400

    emo_inputs = emo_tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        emo_outputs = emo_model(**emo_inputs)
        logits = emo_outputs.logits
        probs = F.softmax(logits, dim=-1)

    top_k = 5
    top_probs, top_indices = torch.topk(probs, k=top_k, dim=-1)

    emotions = []
    for score, idx in zip(top_probs[0], top_indices[0]):
        label = emo_model.config.id2label[idx.item()]
        emotions.append({"label": label, "score": float(score.item())})

    top_emotion = emotions[0]["label"]

    prompt = f"""
    You are a compassionate and empathetic conversational AI.

    User said: "{text}"
    Detected emotion: {top_emotion}

    Task:
    - Reply in first person (using "I" and "me").
    - Your reply MUST be based on validating the user's emotion.
    - Your tone must be warm, supportive, and gentle.
    - If user is happy converse with them hear their story and make conversations with them.
    - If the user's emotion is 'sadness' or 'grief', you can gently suggest talking to people like friends, family, or professionals.
    - Keep the reply short and natural, like a friend would.
    - DO NOT give any other advice, solutions, or suggestions.
    - Focus on acknowledging their feeling and offering to listen.
    - If the emotion is sadness, do NOT use language that could be interpreted as self-harm or despair.
    """

    try:
        gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-05-20')
        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.GenerationConfig(
                temperature=0.7,
                top_p=0.9,
                top_k=40,
                max_output_tokens=100
            )
        )
        reply = response.text
    except Exception as e:
        print(f"API Error: {e}")
        if top_emotion in ['sadness', 'grief']:
            reply = "I hear you. Sometimes it helps to talk to friends or family about what you're going through. I'm here to listen, too."
        else:
            reply = "I hear you. I'm here if you want to talk more about it."

    return jsonify({
        "input": text,
        "emotions": emotions,
        "reply": reply
    })

In [ ]:
try:
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)

    app.run(port=5000, use_reloader=False)
except Exception as e:
    print(f"Failed to create ngrok tunnel or run the app: {e}")

### This is the output that we get once we do an API test.

```{
    "input": "I am really sad right now",
    "emotions": [
        {
            "label": "sadness",
            "score": 0.9742826819419861
        },
        {
            "label": "disappointment",
            "score": 0.006741698365658522
        },
        {
            "label": "grief",
            "score": 0.0031700627878308296
        },
        {
            "label": "neutral",
            "score": 0.002348646055907011
        },
        {
            "label": "remorse",
            "score": 0.0016634233761578798
        }
    ],
    "reply": "I hear you. Sometimes it helps to talk to friends or family about what you're going through. I'm here to listen, too."
}